<a href="https://colab.research.google.com/github/rafbymokodompit/diabetes-ml-project/blob/main/Human_Identity_Recognition_in_Digital_Images_Using_Deep_Learning_Based_Face_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless deepface mtcnn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.0 MB/s eta 0:00:00


In [ ]:
!pip install dlib

In [ ]:
import cv2
from deepface import DeepFace
from mtcnn import MTCNN
import dlib
import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATASET_DIR = '/content/drive/MyDrive/PROJEK IT/Human Identity Recognition in Digital Images Using Deep Learning-Based Face Recognition/Dataset dummy'

In [ ]:
import os

for person in os.listdir(DATASET_DIR):
    person_path = os.path.join(DATASET_DIR, person)
    if os.path.isdir(person_path):
        print(person, "->", len(os.listdir(person_path)), "images")


kit harington -> 5 images
Maisie Williams -> 3 images


In [ ]:
PROCESSED_DIR = '/content/drive/MyDrive/PROJEK IT/Human Identity Recognition in Digital Images Using Deep Learning-Based Face Recognition/Processed Faces'
os.makedirs(PROCESSED_DIR, exist_ok=True)

detector = MTCNN()

In [ ]:
#Loop tiap folder orang
for person_name in os.listdir(DATASET_DIR):
    person_path = os.path.join(DATASET_DIR, person_name)
    save_path = os.path.join(PROCESSED_DIR, person_name)
    os.makedirs(save_path, exist_ok=True)

    for img_name in tqdm(os.listdir(person_path), desc=f"Processing {person_name}"):
        if not img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            pass  # skip file non-image
        else:
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (800, 800))
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                faces = detector.detect_faces(img_rgb)

                if len(faces) > 0:
                  for i, face in enumerate(faces):
                        x, y, w, h = face['box']
                        x, y = max(0, x), max(0, y)
                        face_crop = img_rgb[y:y+h, x:x+w]
                        face_resized = cv2.resize(face_crop, (160, 160))
                        save_img_path = os.path.join(save_path, f"{os.path.splitext(img_name)[0]}_face{i}.jpg")
                        cv2.imwrite(save_img_path, cv2.cvtColor(face_resized, cv2.COLOR_RGB2BGR))
                else:
                    print(f"⚠️ Tidak ada wajah di {img_name}")
            else:
                print(f"❌ Gagal membaca {img_name}")


Processing Maisie Williams: 100%|██████████| 3/3 [00:00<00:00,  3.07it/s]


In [ ]:
!pip install keras-facenet
!pip install mtcnn
!pip install tensorflow


  Preparing metadata (setup.py) ... done
  Created wheel for keras-facenet: filename=keras_facenet-0.3.2-py3-none-any.whl size=10367 sha256=de15b7def61d5e4d830f224698d10c748e0674a85ca5d5d460bc935ccc76fc29
  Stored in directory: /root/.cache/pip/wheels/05/b0/f5/19ac49fedc10b1df3ee56b096edbcfa39d45794fccc6bcdbbf
Successfully built keras-facenet


## Face Embedding Extraction

In [ ]:
from keras_facenet import FaceNet
import numpy as np
import cv2
import os

# Inisialisasi FaceNet
embedder = FaceNet()

# Path hasil crop wajah
CROPPED_DIR = '/content/drive/MyDrive/PROJEK IT/Human Identity Recognition in Digital Images Using Deep Learning-Based Face Recognition/Processed Faces'
# Tempat menyimpan embedding
face_embeddings = []
face_names = []

for person_name in os.listdir(CROPPED_DIR):
    person_path = os.path.join(CROPPED_DIR, person_name)
    if not os.path.isdir(person_path):
        continue

    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Buat embedding
        embedding = embedder.embeddings([img_rgb])
        face_embeddings.append(embedding[0])
        face_names.append(person_name)

face_embeddings = np.array(face_embeddings)
face_names = np.array(face_names)

print(f"✅ Total wajah: {len(face_names)}")
print(f"✅ Bentuk embedding: {face_embeddings.shape}")


1/1 [==============================] - 0s 107ms/step
✅ Total wajah: 4
✅ Bentuk embedding: (4, 512)
